In [1]:
!pip install rank_bm25 -q
!pip install faiss-cpu -q
!pip install sentence-transformers -q
!pip install langchain -q
!pip install peft==0.12.0 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 67.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 8.0 MB/s eta 0:00:00ta 0:00:01


# 文档分块与向量化

In [3]:
# 从.txt文件加载文档
import os

def load_txt_documents(folder_path):
    documents = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path,'r',encoding='utf-8') as file:
                documents.append(file.read())  # 只保留文档内容
    return documents

#测试该函数
files = "/kaggle/input/knowledge-documents"
documents = load_txt_documents(files)  # 大小为 num_of_file * 一大块文档字符串
print(f"共加载了{len(documents)}个文档")

共加载了2个文档


In [4]:
# 文档分块与预处理
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter  
from langchain.schema import Document
from typing import List
import re
    
def split_documents_by_paragraph(documents: List[str], chunk_size: int = 500) -> List[str]:  
    """  
    将文档列表按段完整分块，每段按照 `\n\n` 优先切割，  
    遇到单独的段落超出 chunk_size 时将其划归下一个 chunk。  
    """  
    chunks = []  

    for document in documents:  
        # 遍历每一个文档
        document_cleaned = re.sub(r'[ \t]+', ' ', document.strip()) # 用单个空格代替空格和制表符
        paragraphs = document_cleaned.split("\n\n")  # 按照空行进行段落划分

        # 暂时没有处理一个段落一个chunk装不下的问题（考虑如果装不下需要有合理的overlap部分）
        for paragraph in paragraphs:  
            if paragraph.strip():  
                chunks.append(paragraph.strip())  

    return chunks 
   
# 测试该函数
chunks = split_documents_by_paragraph(documents, chunk_size=500)
print(len(chunks))
# for idx,chunk in enumerate(chunks):
#     print(f"分块{idx+1}:\n{chunk}\n")

244
分块1:
导论 马克思主义中国化时代化的历史进程与理论成果
马克思主义理论的基本特征：马克思主义理论是科学的理论、人民的理论、实践的理论、不断发展的开放的理论

分块2:
一、马克思主义中国话时代化的提出
太平天国运动、洋务运动、戊戌变法、义和团运动、辛亥革命接连而起，但都以失败而告终。各种思潮竞相发声，包括改良主义、自由主义、社会达尔文主义、无政府主义、实用主义、民粹主义、工团主义等“你方唱罢我登场”但都未能解决中国的实际问题。

分块3:
十月革命一声炮响，给中国送来了马克思列宁主义，五四运动（巴黎和会中国外交失败）促进了马克思主义在中国的广泛传播。在马克思列宁主义同中国工人运动的紧密结合中，1921年中国共产党应运而生。

分块4:
找到了马克思主义这个崭新的思想武器，并不意味着就能够自然而然地解决中国革命所面临的问题，还有一个如何把马克思主义基本原理同中国具体实际相结合、同中华优秀传统文化相结合的问题。（城市中心论和农村中心论）

分块5:
1931年开始的在党内占统治地位的“左”倾错误，把马克思主义教条化、把共产国际决议和苏联经验神圣化，使中国革命遭受严重挫折。（湘江战役惨胜。遵义会议确立了毛泽东的领导地位）

分块6:
实现马克思主义中国化。1938年，毛泽东在党的六届六中全会上作了《论新阶段》的报告，强调：“没有抽象的马克思主义，只有具体的马克思主义。……”这标志着“马克思主义的中国化”这一命题的正式提出，并提出毛泽东思想是“中国化的马克思主义”。

分块7:
党的十八大以来，以习近平同志为核心的党中央明确提出要不断推进马克思主义中国化时代化。

分块8:
二、马克思主义中国化时代化的内涵
马克思主义中国化时代化,就是立足中国国情和时代特点，坚持把马克思主义基本原理同中国具体实际相结合、同中华优秀传统文化相结合（“两个结合”的思想）

分块9:
“用马克思主义的文场方法来解决中国问题，创造些新的东西，这样就用得了。”“解决中国问题”和“创造些新的东西”实际上指向马克思主义中国化时代化的科学内涵具体包含三层意思:
1. 运用马克思主义的立场、观点和方法,观察时代、把握时代、引领时代,解决中国革命、建设、改革中的实际问题。
2. 总结和提炼中国革命、建设、改革的实践经验并将其上升为理论，不断丰富和发展马克思主义的理论宝库,赋予马克思主义以新的时代内

In [5]:
from sentence_transformers import SentenceTransformer  

# 加载句向量模型（可替换为其他 embedding 模型）  
embedding_model = SentenceTransformer('all-MiniLM-L6-v2') 

# 考虑使用自己的embedding模型or?

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### 文本向量化

In [ ]:
# 实验函数：检查词向量的相似性
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# 加载预训练模型
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 获取词向量
def get_word_embedding(word):
    inputs = tokenizer(word, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    # 使用最后隐藏层的第一个标记作为词向量
    return outputs.last_hidden_state[0][1].numpy()

# 比较相似性
word1 = "king"
word2 = "queen"
embedding1 = get_word_embedding(word1)
embedding2 = get_word_embedding(word2)

similarity = cosine_similarity([embedding1], [embedding2])
print(f"Cosine similarity between '{word1}' and '{word2}': {similarity[0][0]:.4f}")


### 使用BM2,包含向量化+检索结构

In [30]:
# 根据bm25检索文本内部自动计算
import numpy as np
from collections import Counter
import jieba
from rank_bm25 import BM25Okapi

stopwords = set(["的","是","了","请","简述","论证","回答","解释","说明","一下","什么"])

def create_bm25(chunks, stopwords=set(["的","是","了","请","简述","论证","回答","解释","说明","一下","什么"])
):
    """  
    为文本块创建 BM25 索引和嵌入向量。  
    chunks: 文本块列表。  
    return: BM25 索引。  
    """ 
    # 分词函数
    def preprocess_text(text):
        # 使用 jieba 分词并移除停用词
        return [word for word in jieba.cut(text) if word not in stopwords]

    tokenized_chunks = [preprocess_text(chunk) for chunk in chunks]  
    bm25 = BM25Okapi(tokenized_chunks)  # k1=1.5, b=0.75  # 接受的文档（集合）是分词后的词列表[[],[],[]]
    
    # 对文本向量化储存并且一起返回
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = embedding_model.encode(chunks, convert_to_numpy=True)

    return bm25, embedding_model, embeddings

def bm25_search(query, bm25, stopwords=set(["的","是","了","请","简述","论证","回答","解释","说明","一下","什么"])):
    """  
    使用 BM25 索引查找与用户查询相关的 top_k 文本块。  
    :param query: 用户查询，字符串。  
    :param bm25: 已创建的 BM25 索引。  
    :param chunks: 文本块集合。  
    :param top_k: 返回前 K 个相关文本块。  
    :return: 检索到的文本块及其分数。  
    """
    query_tokens = [word for word in jieba.cut(query) if word not in stopwords]
    scores = bm25.get_scores(query_tokens)
    # ranked_indices = sorted(range(len(scores)), key=lambda i:scores[i],reverse=True)[:top_k]
    # return [(chunks[i],scores[i]) for i in ranked_indices]
    return scores

def similarity_search(query, embedding_model, embeddings):  
    """  
    使用嵌入模型基于语义相似性查找相关文本块。  
    :param query: 用户查询，字符串。  
    :param embedding_model: 已加载的嵌入模型。  
    :param embeddings: 文本块嵌入向量数组。  
    :param chunks: 文本块集合。  
    :param top_k: 返回前 K 个相关文本块。  
    :return: 检索到的文本块及其相似性分数。  
    """  
    def cosine_similarity(vec1, vec2):
        dot_product = np.dot(vec1, vec2)
        norm_vec1 = np.linalg.norm(vec1)
        norm_vec2 = np.linalg.norm(vec2)
        return dot_product / (norm_vec1 * norm_vec2)
    query_embedding = embedding_model.encode(query, convert_to_numpy=True)  
    # 计算余弦相似度  
    similarity_scores = np.array([cosine_similarity(prompt, query_embedding) for prompt in embeddings]) 
    # ranked_indices = np.argsort(similarity_scores)[::-1][:top_k]  
    # return [(chunks[i], similarity_scores[i]) for i in ranked_indices]
    return similarity_scores

def rank_fusion(bm25_score, similarity_score, bm25_weight=0.5, embedding_weight=0.5, top_k=5):  
    """  
    使用排名融合结合 BM25 和语义嵌入的结果。  
    :param bm25_score: BM25 检索结果 [score]。  
    :param similarity_score: 嵌入检索结果 [score]。  
    :param bm25_weight: BM25 分数的权重。  
    :param embedding_weight: 嵌入分数的权重。  
    :param top_k: 返回前 K 个候选文本块。  
    :return: 融合后的文本块及评分。  
    """  
    # 加权求和 BM25 分数和相似度分数  
    scores = bm25_weight*bm25_score + embedding_weight*similarity_score
    # ranked_indices = np.argsort(scores)[::-1][:top_k]  

    # 按照分数排序并返回 top_k 结果 
    # ranked_chunks = [(chunks[i], scores[i]) for i in ranked_indices]
    # return ranked_chunks 
    return scores

# 测试
# user_input = "新民主主义革命的三大法宝"
# user_input = "你好" 
# user_input = "领导人的精髓"
# user_input = "一国两制的基本内容"
# bm25, embedding_model, embeddings = create_bm25(chunks)   # 实际使用时作为chat函数输入
# bm25_scores = bm25_search(user_input, bm25)
# embedding_scores = similarity_search(user_input, embedding_model, embeddings)
# fused_scores = rank_fusion(bm25_scores, embedding_scores)
# top_k = 5
# ranked_indices = np.argsort(fused_scores)[::-1][:top_k]  
# retrieve_chunks = [(chunks[i], fused_scores[i]) for i in ranked_indices]

# #增加一个:过滤分数小于1的结果
# filtered_chunks = [sub_chunk for sub_chunk in retrieve_chunks if sub_chunk[1] > 1]  

# for sub_chunk in filtered_chunks:  
#     print(f"Sub-chunk : \n{sub_chunk[0]}\n(Score: {sub_chunk[1]:.2f})")

# 检索函数包装

In [40]:
def retrieve_text_by_bm25_and_similarity(query_message, bm25, embedding_model, embeddings, chunks,top_k = 5):
    '''
    通过BM25和语义相似度检索文本
    :param query_message: 用户查询字符串。
    :param bm25: 已创建的BM25索引。
    :param embedding_model: 已加载的嵌入模型。
    :param embeddings: 文本块嵌入向量数组。
    :param chunks: 文本块集合。
    :param top_k: 返回前K个相关文本块。
    :return: 检索到的文本块。
    '''
    bm25_scores = bm25_search(query_message, bm25)
    embedding_scores = similarity_search(query_message, embedding_model, embeddings)
    fused_scores = rank_fusion(bm25_scores, embedding_scores)
    ranked_indices = np.argsort(fused_scores)[::-1][:top_k]  
    
    retrieve_texts = []
    for i in ranked_indices:
        retrieve_texts.append((chunks[i], fused_scores[i]))
    filtered_chunks = [(text, score) for text, score in retrieve_texts if score > 1]  
    return filtered_chunks

# 用户查询向量化以及知识库检索

In [ ]:
# 根据bm25检索文本
import numpy as np
from collections import Counter
import jieba
stopwords = set(["的","是","了","请","简述","论证","回答","解释","说明","一下","什么"])

def bm25_search(query, bm25, chunks, top_k=3):
    """  
    使用 BM25 索引查找与用户查询相关的 top_k 文本块。  
    :param query: 用户查询，字符串。  
    :param bm25: 已创建的 BM25 索引。  
    :param chunks: 文本块集合。  
    :param top_k: 返回前 K 个相关文本块。  
    :return: 检索到的文本块及其分数。  
    """
    query_tokens = list(query)
    scores = bm25.get_scores(query_tokens)
    ranked_indices = sorted(range(len(scores)), key=lambda i:scores[i],reverse=True)[:top_k]
    return [(chunks[i],scores[i]) for i in ranked_indices]

def embedding_search(query, embedding_model, embeddings, chunks, top_k=5):  
    """  
    使用嵌入模型基于语义相似性查找相关文本块。  
    :param query: 用户查询，字符串。  
    :param embedding_model: 已加载的嵌入模型。  
    :param embeddings: 文本块嵌入向量数组。  
    :param chunks: 文本块集合。  
    :param top_k: 返回前 K 个相关文本块。  
    :return: 检索到的文本块及其相似性分数。  
    """  
    query_embedding = embedding_model.encode(query, convert_to_numpy=True)  
    similarity_scores = np.dot(embeddings, query_embedding)  # 计算余弦相似度  
    ranked_indices = np.argsort(similarity_scores)[::-1][:top_k]  
    return [(chunks[i], similarity_scores[i]) for i in ranked_indices]

def rank_fusion(bm25_results, embedding_results, bm25_weight=0.5, embedding_weight=0.5, top_k=5):  
    """  
    使用排名融合结合 BM25 和语义嵌入的结果。  
    :param bm25_results: BM25 检索结果 [(chunk, score)]。  
    :param embedding_results: 嵌入检索结果 [(chunk, score)]。  
    :param bm25_weight: BM25 分数的权重。  
    :param embedding_weight: 嵌入分数的权重。  
    :param top_k: 返回前 K 个候选文本块。  
    :return: 融合后的文本块及评分。  
    """  
    scores = {}  
    # 累加 BM25 分数和嵌入分数  
    for chunk, score in bm25_results:  
        scores[chunk] = scores.get(chunk, 0) + score * bm25_weight  
    for chunk, score in embedding_results:  
        scores[chunk] = scores.get(chunk, 0) + score * embedding_weight  

    # 按照分数排序并返回 top_k 结果 
    ranked_chunks = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_k]  
    return ranked_chunks 

def calculate_coverage_score(user_input, chunk):  
    """  
    计算 chunk 覆盖 user_input 的比例（覆盖率）。  
    :param user_input: 用户查询，字符串。  
    :param chunk: 候选 chunk 文本内容，字符串。  
    :return: 覆盖率得分（介于 0 和 1 之间）。  
    """  
    # 分词（可以根据语言选择分词器）  
    user_words = set(jieba.lcut(user_input))
    chunk_words = set(jieba.lcut(chunk))

    user_words = user_words - stopwords
    chunk_words = chunk_words - stopwords

    total_user_words = len(user_words)  
    matched_words = len(user_words & chunk_words)
 
    return matched_words / total_user_words

'''
def filter_chunks(user_input, chunks):
    """  
    对筛选得到的chunks再次进行"\n\n"划分，挑选出每个chunk中覆盖user_input的上下文  
    """  
    relevant_chunks = []
    relevant_scores = []
    for chunk in chunks:
        # print(chunk)
        # print(type(chunk))
        sub_chunks = chunk[0].split("\n\n")   
        for sub_chunk in sub_chunks:  
            score = calculate_coverage_score(user_input, sub_chunk)  
            if score > 0: 
                relevant_chunks.append(sub_chunk)  
                relevant_scores.append(score) 
    
    sorted_chunks_scores = sorted(zip(relevant_chunks, relevant_scores), key=lambda x: x[1], reverse=True)  
    top_chunks_scores = sorted_chunks_scores[:1]  

    # 解压返回分数最高的三个段落和对应分数  
    top_chunks, top_scores = zip(*top_chunks_scores) if top_chunks_scores else ([], [])  

    return list(top_chunks), list(top_scores)
'''

# 测试
user_input = "新民主主义革命的三大法宝"
# user_input = "领导人的精髓"
# user_input = "一国两制的基本内容"
bm25_results = bm25_search(user_input, bm25, chunks)
embedding_results = embedding_search(user_input, embedding_model, embeddings, chunks)
fused_results = rank_fusion(bm25_results, embedding_results)
retrieved_docs = fused_results[:3]
# filtered_results = filter_chunks(user_input, retrieved_docs) # 该函数只返回了最好的一个

test_result = retrieved_docs[0]
sub_chunks = test_result[0]
scores = test_result[1]
for sub_chunk,score in zip(sub_chunks,scores):  
    print(f"Sub-chunk : \n{sub_chunk}\n(Score: {score:.2f})")

In [ ]:
# 根据覆盖率检索文本
from collections import Counter
import jieba

stopwords = set(["的","是","了","请","简述","论证","回答","解释","说明","一下"])

def calculate_coverage_score(user_input, chunk):  
    """  
    计算 chunk 覆盖 user_input 的比例（覆盖率）。  
    :param user_input: 用户查询，字符串。  
    :param chunk: 候选 chunk 文本内容，字符串。  
    :return: 覆盖率得分（介于 0 和 1 之间）。  
    """  
    # 分词（可以根据语言选择分词器）  
    # user_words = user_input.split()  
    # chunk_words = chunk.split()  
    user_words = set(jieba.lcut(user_input))
    chunk_words = set(jieba.lcut(chunk))
    # print(user_words)
    # print(chunk_words)

    # 移除停用词
    user_words = user_words - stopwords
    chunk_words = chunk_words - stopwords

    # 统计用户查询词的数量  
    total_user_words = len(user_words)  
    
    # 计算两个集合的交集  
    # matched_words = len(set(user_words) & set(chunk_words)) 
    matched_words = len(user_words & chunk_words)

    # 覆盖率 = 匹配词数 / 用户输入总词数  
    return matched_words / total_user_words

def retrieve_by_coverage(user_input, chunks, top_k=3):  
    """  
    基于覆盖率检索与用户查询最相关的 top-k 文本块。  
    user_input: 用户查询，字符串。  
    chunks: 文本块集合，列表。  
    top_k: 返回的相关文本块数量。  
    return: 覆盖率最高的文本块列表。  
    """  
    # 计算所有 chunk 的覆盖率得分  
    scores = []  
    for idx, chunk in enumerate(chunks):  
        score = calculate_coverage_score(user_input, chunk)  
        scores.append((chunk, score))  

    # 按覆盖率得分从高到低排序  
    scores = sorted(scores, key=lambda x: x[1], reverse=True)  

    # 返回覆盖率最高的 top-k 文本块  
    return [chunk for chunk, score in scores[:top_k]],[score for chunk, score in scores[:top_k]]

def filter_chunks_by_partial_coverage(user_input, chunks):
    """  
    对筛选得到的chunks再次进行"\n\n"划分，挑选出每个chunk中覆盖user_input的上下文  
    """  
    relevant_chunks = []
    relevant_scores = []
    for chunk in chunks: 
        #print(type(chunk)) # str
        sub_chunks = chunk.split("\n\n")  
        #print(sub_chunks)
        # 每个子段按覆盖率计算是否保留  
        for sub_chunk in sub_chunks:  
            score = calculate_coverage_score(user_input, sub_chunk)  
            if score > 0:  # 只保留覆盖了用户查询内容的部分  
                relevant_chunks.append(sub_chunk)  
                relevant_scores.append(score)
    # return relevant_chunks, relevant_scores
    sorted_chunks_scores = sorted(zip(relevant_chunks, relevant_scores), key=lambda x: x[1], reverse=True)  
    top_chunks_scores = sorted_chunks_scores[:3]  

    # 解压返回分数最高的三个段落和对应分数  
    top_chunks, top_scores = zip(*top_chunks_scores) if top_chunks_scores else ([], [])  

    return list(top_chunks), list(top_scores)


user_query = "新民主主义革命的三大法宝"
# user_query = "请简述一下一国两制的基本内容"
# user_query = "中国历代领导人的精髓是什么"
retrieved_chunks,scores = retrieve_by_coverage(user_query, chunks, top_k=3) 
filtered_chunks,filtered_scores = filter_chunks_by_partial_coverage(user_query, retrieved_chunks)

# print("覆盖率最高的 chunk 整体内容：")  
# for idx, chunk in enumerate(retrieved_chunks):  
#     print(f"Chunk {idx + 1}: {chunk} (Score: {scores[idx]:.2f})\n")  

print("\n覆盖相关内容的最简上下文：")  
for idx, sub_chunk in enumerate(filtered_chunks):  
    print(f"Sub-chunk {idx + 1}: {sub_chunk}(Score: {filtered_scores[idx]:.2f})\n")  

In [ ]:
# 根据向量距离检索文本

# 构建向量检索FAISS索引，方便后续检索  
import faiss  
import numpy as np  

def create_faiss_index(embeddings, chunks):  
    """  
    embeddings: 文本块向量表示 (numpy array)。  
    chunks: 文本快列表，用于后续根据索引ID查询原始内容。  
    """  
    # L2 距离的索引 
    # dim = embeddings.shape[1]  
    # index = faiss.IndexFlatL2(dim)   

    # 余弦相似度
    embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
    index = faiss.IndexFlatIP(embeddings.shape[1])
    
    index.add(embeddings)

    # 可选：存储块内容 （id -> chunk）  
    id_to_chunk = {i: chunk for i, chunk in enumerate(chunks)}  
    return index, id_to_chunk

# 用户查询向量检索
def search_top_k(query, index, model, id_to_chunk, top_k=3):  
    """  
    query: 用户输入的查询。  
    index: FAISS 索引对象。  
    model: 句向量生成模型。  
    id_to_chunk: FAISS ID -> 文本块映射。  
    top_k: 返回的相关文本块数量。  
    return: 检索到的相关文本块列表。  
    """  
    query_embedding = model.encode([query], convert_to_numpy=True)  # 查询向量化  
    distances, indices = index.search(query_embedding, top_k)       # FAISS 检索 
    for distance,idx in zip(distances, indices):
        print(distance,idx)
    
    return [id_to_chunk[i] for i in indices[0] if i in id_to_chunk]

# 测试该函数
# faiss_index, id_to_chunk = create_faiss_index(embeddings, chunks)
# print(faiss_index)
# user_query = "新民主主义革命的三大法宝"  
# retrieved_chunks = search_top_k(user_query, faiss_index, embedding_model, id_to_chunk, top_k=3) 

# 和我们的聊天机器人进行整合

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

model_path = "/kaggle/input/qwen2.5/transformers/3b/1"
lora_dir = "/kaggle/input/lora-3b/transformers/default/1"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = PeftModel.from_pretrained(model, lora_dir)
model.to(device)
model.eval()

In [ ]:
max_input_length = model.config.max_position_embeddings  

def chat_with_bm25():  
    
    print("开始聊天（输入 '\\quit' 结束对话，输入 '\\newsession' 开启新会话）：")  
    system_prompt = "你是一名助手，请根据用户问题以及提供的相关文档，提炼出你的答案，并对用户进行回答。"
    chat_history = [{"role": "system", "content": system_prompt}] 
    
    while True:  
        user_input = input("用户：")  
        if user_input.lower() == "\\quit":  
            print("结束对话。")  
            break  

        elif user_input.lower() == "\\newsession":  
            print("开启新会话。")  
            chat_history = []  
            i = 0
            continue 

        # 检索相关文档
        bm25_results = bm25_search(user_input, bm25, chunks)
        embedding_results = embedding_search(user_input, embedding_model, embeddings, chunks)
        fused_results = rank_fusion(bm25_results, embedding_results)
        retrieved_docs = fused_results[:3]
        filtered_results = filter_chunks(user_input, retrieved_docs)
        sub_chunks = filtered_results[0]
        print(filtered_results)
        context = "\n".join(sub_chunks) # 将检索到的文档合并为上下文
        #print(context)
        prompt = f"相关文档:\n{context}用户问题: {user_input}\n\n答案:"
        # prompt = f"""
        #             Give the answer to the user query delimited by triple backticks ```{user_input}```\
        #             using the information given in context delimited by triple backticks ```{context}```.\
        #             Be concise and output the answer of size less than 200 tokens.
        #             """
        chat_history.append({"role": "user", "content": prompt})
        
        text = tokenizer.apply_chat_template(chat_history, tokenize=False, add_generation_prompt=True)
        model_inputs = tokenizer([text], return_tensors="pt").to(device)
        while len(model_inputs["input_ids"]) > max_input_length:  
            chat_history.pop(1)
            chat_history.pop(1)
            text = tokenizer.apply_chat_template(chat_history, tokenize=False, add_generation_prompt=True)
            model_inputs = tokenizer([text], return_tensors="pt")
            if len(model_inputs["input_ids"]) <= max_input_length:
                break
        
        generated_ids = model.generate(input_ids=model_inputs["input_ids"],
                                       attention_mask=model_inputs["attention_mask"],
                                       pad_token_id=tokenizer.pad_token_id,
                                       max_new_tokens=512,
                                       do_sample=True,
                                       repetition_penalty=1.2,  # 惩罚重复生成
                                       temperature=0.7)  

        generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        
        print(f"助手： {response}")  
        chat_history.append({"role": "assistant", "content": response})
        

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

model_path = "/kaggle/input/qwen2.5/transformers/3b/1"
lora_dir = "/kaggle/input/lora-3b/transformers/default/1"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = PeftModel.from_pretrained(model, lora_dir)
model.to(device)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 2048)
        (layers): ModuleList(
          (0-35): 36 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [41]:
# 准备知识库

files = "/kaggle/input/knowledge-documents"
documents = load_txt_documents(files)
chunks = split_documents_by_paragraph(documents, chunk_size=500)
bm25, embedding_model, embeddings = create_bm25(chunks) 

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

In [95]:
max_input_length = model.config.max_position_embeddings  

def chat_with_bm25(bm25, chunks, max_input_length, model, tokenizer, device):  
    
    print("开始聊天（输入 '\\quit' 结束对话，输入 '\\newsession' 开启新会话）：")  
    system_prompt = "你是一名助手，请根据用户问题以及提供的相关文档，提炼出你的答案，并对用户进行回答，如果没有相关文档，就请你自己对用户问题进行回答。"
    chat_history = [{"role": "system", "content": system_prompt}] 
    
    while True:  
        user_input = input("用户：")  
        if user_input.lower() == "\\quit":  
            print("结束对话。")  
            break  

        elif user_input.lower() == "\\newsession":  
            print("开启新会话。")  
            chat_history = [{"role": "system", "content": system_prompt}]  
            i = 0
            continue 

        # 检索相关文档
        context = retrieve_text_by_bm25_and_similarity(user_input, bm25, embedding_model, embeddings, chunks,top_k = 3)  # 会将得分最高的k个文档合并返回,k=1效果会更好
        #context = '\n'.join(text for text, score in context)
        context = '\n'.join(text.replace('\n', ', ') for text, score in context) # 如果一段上下文里有换行,则将其用逗号拼接为一个长句
        if not context.strip():  # 使用 strip() 检查空白  
            context = "无"
        print(f"检索到的相关文档：\n {context}\n") 
        #prompt = f"相关文档:{context}\n用户问题: {user_input}\n\n"
        prompt = f"下面是与用户问题相关的文档内容:\n{context}\n\n请根据这些文档内容回答以下问题:\n用户问题: {user_input}\n"
        #prompt = f"请回答以下问题:\n用户问题: {user_input}\n你可以参考文档回答，下面是与用户问题相关的文档内容:\n{context}\n\n"
        chat_history.append({"role": "user", "content": prompt})
        
        text = tokenizer.apply_chat_template(chat_history, tokenize=False, add_generation_prompt=True)
        model_inputs = tokenizer([text], return_tensors="pt").to(device)
        while len(model_inputs["input_ids"]) > max_input_length:  
            chat_history.pop(1)
            chat_history.pop(1)
            text = tokenizer.apply_chat_template(chat_history, tokenize=False, add_generation_prompt=True)
            model_inputs = tokenizer([text], return_tensors="pt")
            if len(model_inputs["input_ids"]) <= max_input_length:
                break
        
        generated_ids = model.generate(input_ids=model_inputs["input_ids"],
                                       attention_mask=model_inputs["attention_mask"],
                                       pad_token_id=tokenizer.pad_token_id,
                                       max_new_tokens=512,
                                       do_sample=True,
                                       repetition_penalty=1.2, 
                                       temperature=0.7)  

        generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        
        print(f"助手： {response}")  
        chat_history.append({"role": "assistant", "content": response})
        

In [96]:
# run
print("初始化检索数据库...")
chat_with_bm25(bm25, chunks, max_input_length, model, tokenizer, device)

初始化检索数据库...
开始聊天（输入 '\quit' 结束对话，输入 '\newsession' 开启新会话）：


用户： 你好


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

检索到的相关文档：
 无

助手： 您好！有什么可以帮助您的吗？


用户： 新民主主义革命的三大法宝是什么


...

用户： 感谢你的回答


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

检索到的相关文档：
 一、马克思主义中国话时代化的提出, 太平天国运动、洋务运动、戊戌变法、义和团运动、辛亥革命接连而起，但都以失败而告终。各种思潮竞相发声，包括改良主义、自由主义、社会达尔文主义、无政府主义、实用主义、民粹主义、工团主义等“你方唱罢我登场”但都未能解决中国的实际问题。

助手： 不用客气，随时为您解答疑惑。


用户： \quit


结束对话。


In [ ]:
# 待完成：分层版本(bm25+覆盖率+向量距离))
# chat_with_bm25
# chat_with_docs